<a href="https://colab.research.google.com/github/gaby-chu/DermaLLM/blob/main/product_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
product_data = pd.read_csv('/content/drive/MyDrive/aml/product_info.csv')

In [ ]:
product_data.columns

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [ ]:
product_data.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


In [ ]:
# check for any missing product names
product_data['product_name'].isna().sum()

np.int64(0)

In [ ]:
# check for any missing brand names
product_data['brand_name'].isna().sum()

np.int64(0)

In [ ]:
skincare_products = product_data[product_data['primary_category'] == "Skincare"]
#skincare_products['secondary_category'].unique()
skincare_products[skincare_products['secondary_category'] == "Wellness"]['tertiary_category'].unique()

array(['Facial Rollers', 'Beauty Supplements', 'Holistic Wellness'],
      dtype=object)

In [ ]:
product_data['primary_category'].unique()

array(['Fragrance', 'Bath & Body', 'Mini Size', 'Hair', 'Makeup',
       'Skincare', 'Tools & Brushes', 'Men', 'Gifts'], dtype=object)

In [ ]:
product_data[product_data['primary_category'] == "Mini Size"]['secondary_category'].unique()

array(['Fragrance', 'Skincare', 'Hair', 'Makeup', 'Value & Gift Sets',
       'Bath & Body', nan], dtype=object)

In [ ]:
product_data['secondary_category'].unique()

array(['Value & Gift Sets', 'Women', 'Body Moisturizers', 'Fragrance',
       'Men', 'Candles & Home Scents', 'Body Care',
       'Hair Styling & Treatments', 'Shampoo & Conditioner', 'Mini Size',
       'Lip', 'Moisturizers', 'Treatments', 'Eye Care',
       'Lip Balms & Treatments', 'Sunscreen', 'Cleansers', 'Masks',
       'Skincare', 'Face', 'Eye', 'Cheek', 'Hair', 'Tools',
       'Brushes & Applicators', 'Makeup Palettes', 'Beauty Tools',
       'Accessories', 'Makeup', 'Bath & Shower', 'Wellness',
       'High Tech Tools', 'Shop by Concern', nan, 'Self Tanners',
       'Shaving', 'Hair Tools', 'Nail', 'Beauty Accessories',
       'Beauty Supplements', 'Other Needs', 'Bath & Body'], dtype=object)

In [ ]:
## filter dataframe to only have skincare products -- using primary and secondary cateogory tags
product_data = product_data[(product_data['primary_category']=='Skincare') | (product_data['secondary_category']=="Skincare")]
product_data.reset_index(drop = True, inplace = True)
print("Total number of rows:", len(product_data))
product_data.head()

Total number of rows: 2518


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P439055,GENIUS Sleeping Collagen Moisturizer,6018,Algenist,33910,4.5413,1321.0,2 oz/ 60 mL,Size,2 oz/ 60 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Colla...",Skincare,Moisturizers,Moisturizers,0,NaN,NaN
1,P421277,GENIUS Liquid Collagen Serum,6018,Algenist,67870,4.0259,1159.0,1 oz / 30 mL,Size,1 oz / 30 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Colla...",Skincare,Treatments,Face Serums,1,25.0,25.0
2,P467602,Triple Algae Eye Renewal Balm Eye Cream,6018,Algenist,17890,4.5306,1142.0,NaN,NaN,NaN,...,0,0,1,NaN,Skincare,Eye Care,Eye Creams & Treatments,0,NaN,NaN
3,P432045,GENIUS Liquid Collagen Lip Treatment,6018,Algenist,44448,3.8721,649.0,.5 oz / 15 mL,Size,.5 oz / 15 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Plump...",Skincare,Lip Balms & Treatments,NaN,0,NaN,NaN
4,P311143,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,6018,Algenist,27278,4.4134,508.0,1 oz,Size,1 oz,...,0,0,0,"['Vegan', 'Hypoallergenic', 'UV Protection', '...",Skincare,Sunscreen,Face Sunscreen,0,NaN,NaN


In [ ]:
## some products have variations in sizes, types, colors, etc.
## these products have different min and max prices based on the variations

variations = product_data[product_data['child_count'] > 0]
print("Total number of products with variations:", len(variations))

## because people would typically filter on "less than" a certain price, let's use the min child as the price if available
## otherwise, use price_usd as listed since there were some inconsistencies with the price_usd column

product_data['effective_price'] = product_data['child_min_price'].fillna(product_data['price_usd'])

Total number of products with variations: 410


In [ ]:
product_data['highlights'].unique()

array(["['Vegan', 'Good for: Loss of firmness', 'Collagen', 'Hypoallergenic', 'Without Parabens', 'Best for Dry, Combo, Normal Skin']",
       nan,
       "['Vegan', 'Good for: Loss of firmness', 'Plumping', 'Collagen', 'Hypoallergenic', 'Best for Dry, Combo, Normal Skin']",
       ...,
       "['Vegan', 'Hyaluronic Acid', 'Clean at Sephora', 'Hydrating', 'Good for: Dryness', 'Best for Dry, Combo, Normal Skin']",
       "['Vegan', 'Good for: Pores', 'Clean + Planet Positive', 'Good for: Acne/Blemishes', 'Best for Oily, Combo, Normal Skin', 'Without Sulfates SLS & SLES']",
       "['Vegan', 'Good for: Dullness/Uneven Texture', 'Vitamin C', 'Plumping', 'Clean at Sephora', 'Hydrating']"],
      dtype=object)

In [ ]:
## trying to get all the useful information out of the highlights column
## first let's clean it up - lowercase and split by comma/semicolon
import ast

def parse_highlights(x):
    if pd.isna(x):
        return []
    # Try to interpret as list
    try:
        if x.strip().startswith("["):
            return [h.lower().strip() for h in ast.literal_eval(x)]
    except:
        pass
    # Otherwise split by comma or semicolon
    return [h.lower().strip() for h in re.split(",|;", x)]

# Apply to your dataframe
product_data['highlight_list'] = product_data['highlights'].apply(parse_highlights)
product_data['highlight_list'].head()

,highlight_list
0,"[vegan, good for: loss of firmness, collagen, ..."
1,"[vegan, good for: loss of firmness, collagen, ..."
2,[]
3,"[vegan, good for: loss of firmness, plumping, ..."
4,"[vegan, hypoallergenic, uv protection, spf, be..."


In [ ]:
## figuring out ALL unique values inside the highlight list column

# Flatten all lists into one big list
all_highlights = [item for sublist in product_data['highlight_list'] for item in sublist]

# Get unique values
unique_highlights = sorted(set(all_highlights))
count = 0
print("All unique highlights:")
for h in unique_highlights:
    print(h)
    count+=1

print(count)


All unique highlights:
aha/glycolic acid
alcohol free
all hair types
allure 2017 best of beauty award winner
allure 2018 best of beauty award winner
allure 2019 best of beauty award winner
allure 2020 best of beauty award winner
allure 2021 best of beauty award winner
allure 2021: clean beauty winner
allure 2022 best of beauty award winner
best for combination skin
best for dry skin
best for dry, combo, normal skin
best for normal skin
best for oily skin
best for oily, combo, normal skin
black owned at sephora
cbd
clean + planet positive
clean at sephora
collagen
community favorite
cream formula
cruelty-free
customizable
floral scent
fragrance free
fresh scent
full coverage
gluten free
good for: acne/blemishes
good for: anti-aging
good for: damage
good for: dark circles
good for: dark spots
good for: dryness
good for: dullness/uneven texture
good for: loss of firmness
good for: pores
good for: redness
high shine finish
hyaluronic acid
hydrating
hypoallergenic
increases shine
lactic aci

In [ ]:
# which highlight tags appear the most
from collections import Counter

highlight_counts = Counter(all_highlights)

print("Most common highlights:")
for h, count in highlight_counts.most_common(100):
    print(h, count)


Most common highlights:
good for: dullness/uneven texture 773
clean at sephora 763
vegan 710
good for: dryness 619
cruelty-free 602
good for: anti-aging 591
without parabens 440
hydrating 440
hyaluronic acid 407
good for: loss of firmness 402
best for dry, combo, normal skin 390
good for: pores 344
clean + planet positive 317
best for oily, combo, normal skin 300
fragrance free 296
vitamin c 223
without sulfates sls & sles 222
good for: acne/blemishes 208
good for: dark circles 207
good for: dark spots 165
aha/glycolic acid 155
niacinamide 143
good for: redness 142
salicylic acid 133
gluten free 119
without phthalates 114
plumping 109
spf 106
best for dry skin 97
oil free 96
retinol 88
community favorite 74
alcohol free 67
without silicones 63
without mineral oil 57
lactic acid 56
hypoallergenic 54
collagen 49
refill available 44
reef safe spf 41
black owned at sephora 37
radiant finish 35
best for oily skin 34
allure 2020 best of beauty award winner 32
best for normal skin 31
natural 

In [ ]:
## combining some of categories so that the dataset will be less sparse if I make each tag a column

# 1. all "allure 20xx best of beauty award winner" will be combined
# 1. combine all fragrance types to "has fragrance"

# Example highlight mapping: highlights -> column names
highlight_mapping = {
    # general tags
    "vegan": "vegan",
    "cruelty-free": "cruelty_free",
    "without parabens": "paraben_free",
    "without sulfates sls & sles": "sulfate_free",
    "without silicones": "silicone_free",
    "without mineral oil": "mineral_free",

    "fragrance free": "fragrance_free",
    "fresh scent": "has_fragrance",
    "floral scent": "has_fragrance",
    "woody & earthy scent": "has_fragrance",
    "warm &spicy scent": "has_fragrance",
    "unisex/ genderless scent": "has_fragrance",
    "oil free": "oil_free",
    "clean at sephora": "clean_at_sephora",
    "non-comedogenic": "non_comedogenic",

    # skin concerns
    "good for: dullness/uneven texture": "dullness",
    "good for: dryness": "dryness",
    "good for: anti-aging": "anti_aging",
    "good for: acne/blemishes": "acne",
    "good for: dark spots": "dark_spots",
    "good for: redness": "redness",
    "plumping": "plumping",

    # actives / functional ingredients
    "hyaluronic acid": "hyaluronic_acid",
    "niacinamide": "niacinamide",
    "retinol": "retinol",
    "salicylic acid": "salicylic_acid",
    "aha/glycolic acid": "aha_glycolic_acid",
    "spf": "spf",
    "reef safe spf": "spf",
    "alcohol free": "alcohol_free",
    "hypoallergenic": "hypoallergenic"

}

# want to add information about skin types and give mappings as well

# apply the mapping
for highlight, col_name in highlight_mapping.items():
    product_data[col_name] = product_data['highlight_list'].apply(
    lambda hl: int(any(highlight.lower() in h.lower() for h in hl)))


# one of the highlight features is if the product won the allure best of beauty award, but there are several years for this
# combine all "allure 20xx best of beauty" into one column
def check_allure_best_of_beauty(highlights):
    return int(any("allure" in h.lower() and "best of beauty" in h.lower() for h in highlights))

product_data['allure_best_of_beauty'] = product_data['highlight_list'].apply(check_allure_best_of_beauty)


SKIN_TYPE_MAPPING = {
    "best for oily skin": ["oily_skin"],
    "best for dry skin": ["dry_skin"],
    "best for normal skin": ["normal_skin"],
    "best for combination skin": ["combination_skin"],
    "best for oily, combo, normal skin": ["oily_skin", "combination_skin", "normal_skin"],
    "best for dry, combo, normal skin": ["dry_skin", "combination_skin", "normal_skin"]
}
SKIN_TYPE_COLUMNS = ["oily_skin", "dry_skin", "normal_skin", "combination_skin"]

def apply_skin_types(highlights):
    flags = {col: 0 for col in SKIN_TYPE_COLUMNS}

    for h in highlights:
        h_lower = h.lower()
        for pattern, cols in SKIN_TYPE_MAPPING.items():
            if pattern in h_lower:
                for col in cols:
                    flags[col] = 1

    return pd.Series(flags)

skin_type_df = product_data['highlight_list'].apply(apply_skin_types)
product_data = pd.concat([product_data, skin_type_df], axis=1)


# let's also add a column here for sensitive skin to help querying
## I will define products for sensitive skin as ones that are hypoallergenic and fragrance free

product_data['sensitive_skin'] = (
    (product_data['hypoallergenic'] == 1) | (product_data['fragrance_free'] == 1)).astype(int)


In [ ]:
# checking that no product is listed as fragrance free and has fragrance
(product_data['fragrance_free'] & product_data['has_fragrance']).sum()

np.int64(0)

In [ ]:
# check that skin types are populated
product_data[SKIN_TYPE_COLUMNS]

,oily_skin,dry_skin,normal_skin,combination_skin
0,0,1,1,1
1,0,1,1,1
2,0,0,0,0
3,0,1,1,1
4,0,1,1,1
...,...,...,...,...
2513,0,0,0,0
2514,0,1,1,1
2515,0,0,0,0
2516,1,0,1,1


In [ ]:
product_data.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,aha_glycolic_acid,spf,alcohol_free,hypoallergenic,allure_best_of_beauty,oily_skin,dry_skin,normal_skin,combination_skin,sensitive_skin
0,P439055,GENIUS Sleeping Collagen Moisturizer,6018,Algenist,33910,4.5413,1321.0,2 oz/ 60 mL,Size,2 oz/ 60 mL,...,0,0,0,1,0,0,1,1,1,1
1,P421277,GENIUS Liquid Collagen Serum,6018,Algenist,67870,4.0259,1159.0,1 oz / 30 mL,Size,1 oz / 30 mL,...,0,0,0,1,0,0,1,1,1,1
2,P467602,Triple Algae Eye Renewal Balm Eye Cream,6018,Algenist,17890,4.5306,1142.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,P432045,GENIUS Liquid Collagen Lip Treatment,6018,Algenist,44448,3.8721,649.0,.5 oz / 15 mL,Size,.5 oz / 15 mL,...,0,0,0,1,0,0,1,1,1,1
4,P311143,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,6018,Algenist,27278,4.4134,508.0,1 oz,Size,1 oz,...,0,0,0,1,0,0,1,1,1,1


In [ ]:
## create a new column "product text" for embeddings
## goal: this should be compact product description that we can use when the user queries
## it will include: product name, brand, skin concerns it targets, active ingredients, highlight tags, price, and rating

# step 1: define column groups
SKIN_CONCERNS = [
    'dullness', 'dryness', 'anti_aging', 'acne',
    'dark_spots', 'redness', 'plumping'
]

ACTIVES = [
    'hyaluronic_acid', 'niacinamide', 'retinol',
    'salicylic_acid', 'aha_glycolic_acid', 'spf'
]

SAFETY_PREFS = [
    'vegan', 'cruelty_free', 'paraben_free',
    'sulfate_free', 'silicone_free', 'mineral_free',
    'alcohol_free', 'hypoallergenic', 'clean_at_sephora'
]

FORMULATION = [
    'oil_free', 'non_comedogenic'
]

# build the function to create the column

def build_product_text(row):
    parts = []

    # 1. Name + brand
    parts.append(f"{row['product_name']} by {row['brand_name']}.")

    # 2. Skin concerns
    concerns = [c.replace("_", " ") for c in [
        'dullness', 'dryness', 'anti_aging',
        'acne', 'dark_spots', 'redness', 'plumping'
    ] if row.get(c) == 1]

    if concerns:
        parts.append(f"Targets {', '.join(concerns)}.")

    # 3. Actives
    actives = [a.replace("_", " ") for a in [
        'hyaluronic_acid', 'niacinamide', 'retinol',
        'salicylic_acid', 'aha_glycolic_acid', 'spf'
    ] if row.get(a) == 1]

    if actives:
        parts.append(f"Contains {', '.join(actives)}.")

    # 4. Safety & preferences
    prefs = [p.replace("_", " ") for p in [
        'vegan', 'cruelty_free', 'paraben_free',
        'sulfate_free', 'silicone_free', 'mineral_free',
        'alcohol_free', 'hypoallergenic', 'clean_at_sephora'
    ] if row.get(p) == 1]

    if prefs:
        parts.append(f"{', '.join(prefs)}.")

    # 5. Fragrance logic
    if row.get('fragrance_free') == 1:
        parts.append("Fragrance free.")
    elif row.get('has_fragrance') == 1:
        parts.append("Contains fragrance.")

    # 6. Formulation
    formulation = [f.replace("_", " ") for f in [
        'oil_free', 'non_comedogenic'
    ] if row.get(f) == 1]

    if formulation:
        parts.append(f"{', '.join(formulation)}.")

    # 7. Awards
    if row.get('allure_best_of_beauty') == 1:
        parts.append("Allure Best of Beauty award winner.")

    # 8. Price and rating
    if pd.notna(row.get('price_usd')):
        parts.append(f"Price {row['price_usd']} dollars.")
    if pd.notna(row.get('rating')):
        parts.append(f"Rating {row['rating']}.")

    # 9. Skin Types
    skin_types = [st.replace("_", " ") for st in SKIN_TYPE_COLUMNS if row.get(st) == 1]
    if skin_types:
      parts.append(f"Best for {', '.join(skin_types)}.")

    if row.get('sensitive_skin') == 1:
      parts.append("Suitable for sensitive skin.")

    # 10. Child
    if pd.notna(row.get('size')):
      parts.append(f"Size {row['size']}.")


    return " ".join(parts)


  # apply the function
product_data['product_text'] = product_data.apply(build_product_text, axis=1)


In [ ]:
# check if the product_text function worked
for text in product_data['product_text'].tail(5):
    print(text)
    print("--")


The Youth Vault: 13-Piece Vegan Skincare + Apparel Set by Youth To The People. Targets anti aging, acne. Contains retinol, aha glycolic acid. clean at sephora. Price 340.0 dollars. Rating 4.875.
--
Youth Stacks: Plump It Up for Dry, Dehydrated Skin by Youth To The People. Targets dryness. Contains hyaluronic acid. vegan, clean at sephora. Price 28.0 dollars. Rating 4.1667. Best for dry skin, normal skin, combination skin.
--
Mini Mandelic Acid + Superfood Unity Exfoliant by Youth To The People. Targets acne. Contains salicylic acid. alcohol free. Price 16.0 dollars. Rating 4.8. Size 1 oz / 30 mL.
--
Youth Stacks: Daily Skin Health Your Way for Pores and Oiliness by Youth To The People. Targets acne. vegan, sulfate free. Price 20.0 dollars. Rating 4.25. Best for oily skin, normal skin, combination skin.
--
Youth Stacks: Brighter Tomorrow Duo for Dullness by Youth To The People. Targets dullness, plumping. vegan, clean at sephora. Price 28.0 dollars. Rating 3.0.
--


In [ ]:
# checking accuracy with product: The Youth Vault: 13-Piece Vegan Skincare + Apparel Set
product_data.iloc[-5][product_data.iloc[-5] == 1]

,2513
limited_edition,1
online_only,1
sephora_exclusive,1
clean_at_sephora,1
anti_aging,1
acne,1
retinol,1
aha_glycolic_acid,1


In [ ]:
review_data = pd.read_csv('/content/drive/MyDrive/aml/reviews_0-250.csv', on_bad_lines='skip', engine='python')

In [ ]:
review_data.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [ ]:
## add information from reviews into the product_text column

In [ ]:
## start embedding product_text
## using huggingface sentencetransformers

#!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer

# load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


# calculate embeddings by calling model.encode()
embeddings = model.encode(product_data['product_text'])
print(embeddings.shape)
# [2518, 384] - 2518 products, each embedding is 384 dimensions

(2518, 384)


In [ ]:
## check embedding example
embeddings[0]

array([-6.63465485e-02,  1.54168643e-02,  8.70341901e-03,  5.72954528e-02,
        4.99715358e-02, -2.64327563e-02,  4.86885495e-02,  8.79154205e-02,
       -4.62830923e-02, -2.29657497e-02, -8.97181220e-03, -3.86459025e-04,
       -6.21518977e-02,  4.69215661e-02, -8.11096206e-02,  4.33445014e-02,
        6.01364262e-02,  4.47050948e-03,  3.58194262e-02, -5.87633736e-02,
        4.43320125e-02, -1.00092394e-02,  8.18414390e-02,  3.34194116e-02,
        2.04688814e-02, -5.99179342e-02,  6.10520132e-03, -3.80472578e-02,
        2.98931077e-02, -2.01614923e-03,  3.59822810e-02, -2.05349121e-02,
       -1.07907824e-01, -5.50901294e-02,  6.01913370e-02, -4.34429720e-02,
       -2.80370153e-02, -2.67361589e-02, -6.58942685e-02,  8.07427093e-02,
        6.90490827e-02, -5.81903122e-02, -9.46704149e-02,  2.07351688e-02,
        7.38692358e-02, -3.68184783e-02,  6.10851794e-02,  3.73676009e-02,
       -2.72124857e-02,  5.58119267e-02, -5.28167337e-02, -1.11866206e-01,
       -8.05483013e-03, -

In [ ]:
## build retrieval layer to respond to user queries

# using sklearn cosine similarity
from sklearn.metrics.pairwise import cosine_similarity


## need to build query embedding
similarity_scores = cosine_similarity(query_embedding, embeddings)

In [ ]:
## what we are going to return

# top 10 products with the highest cosine similarity scores between query embedding and product text embeddings

top_k = 10
top_indices = np.argsort(similarity_scores[0])[::-1][:top_k]

# Retrieve top products
recommended_products = product_data.iloc[top_indices]

In [ ]:
## we can also use filtering before/after using similarity scores

#filtered_products = recommended_products[
 #   (recommended_products['price_usd'] <= 30) &
   # (recommended_products['oily_skin'] == 1) &
  #  (recommended_products['fragrance_free'] == 1)]